In [50]:
from binance.client import Client
import pandas as pd
import plotly.graph_objects as go
from pon import api,secret
import numpy as np

In [47]:
def get_price_data_binance(ticker:str,interval:Client.KLINE_INTERVAL_1DAY,limit:int)->pd.DataFrame:
    client = Client(api, secret)
    interval = Client.KLINE_INTERVAL_1DAY
    df = pd.DataFrame(client.futures_klines(symbol=ticker, interval=interval, limit=limit))
    df.columns=['date','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df = df.drop(['close_time','d1','d2','d3','d4','d5'],axis=1)
    df['date'] = pd.to_datetime(df['date']*1000000)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    return df


In [71]:
def get_list_of_up_imbalances(df:pd.DataFrame)->list:
    up_imbalances = []
    for i in range(2, len(df)):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        if prev_candle['high'] < next_candle['low']:
            up_imbalances.append([round((prev_candle['high']+next_candle['low'])/2,2), next_candle['low'],df.iloc[i-1]['date']])
    return up_imbalances

def get_list_of_down_imbalances(df:pd.DataFrame)->list:
    down_imbalances = []
    for i in range(2, len(df)):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        if prev_candle['low'] > next_candle['high']:
            down_imbalances.append([round((prev_candle['low']+next_candle['high'])/2,2) , next_candle['high'] , df.iloc[i-1]['date']])
    return down_imbalances

def delete_up_imbalance(up_imbalances, df:pd.DataFrame)->list:
    for imbalance in up_imbalances:
        for i in range(0, len(df)):
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and candle['open'] >= imbalance[0] and (candle['close'] < imbalance[0] or candle['low'] < imbalance[0]):
                print('upbooba')
    up_imbalances.remove(imbalance)
    return up_imbalances

def delete_down_imbalance(down_imbalances, df:pd.DataFrame)->list:
    for imbalance in down_imbalances:
        for i in range(0, len(df)):
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and candle['open'] <= imbalance[0] and (candle['close'] > imbalance[0] or candle['low'] > imbalance[0]):
                print('downbooba')
    down_imbalances.remove(imbalance)
    return down_imbalances

In [42]:
df = get_price_data_binance('BTCUSDT',Client.KLINE_INTERVAL_1DAY,200)

In [72]:
up_imbalances = get_list_of_up_imbalances(df)
down_imbalances = get_list_of_down_imbalances(df)
up_imbalances = delete_up_imbalance(up_imbalances,df)
up_imbalances = delete_down_imbalance(down_imbalances,df)

upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
upbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba
downbooba


In [45]:
fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
            
for imbalance in up_imbalances:
    fig.add_trace(go.Scatter(x=[imbalance[2]], y=[imbalance[0]]))
for imbalance in down_imbalances:
    fig.add_trace(go.Scatter(x=[imbalance[2]], y=[imbalance[0]]))
fig.update_layout(showlegend=False)
fig.show()